### Saved Model - Format

In [ ]:
#!pip install pyyaml h5py

In [1]:
import os
import sys
import yaml
import tensorflow as tf
from tensorflow import keras

2024-04-23 10:36:55.346629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 10:36:55.346670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 10:36:55.347597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 10:36:55.352134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 10:36:56.049382: W tensorflow/compiler/tf2

In [2]:
from model.dataset import Dataset
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [3]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [4]:
config:str = "default" 
cfg = load_config(config)

cli: Configuration from ./config/default.yaml


In [5]:
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])

2024-04-23 10:37:24.258765: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 10:37:24.258936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 10:37:24.288347: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [9]:
from model.trainer import build_fp

In [21]:
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer

In [29]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    #return m_pre, m_specaug, m_fp
    return m_pre, m_specaug, m_fp

In [30]:
m_pre, m_specaug, m_fp = build_fp(cfg)

In [28]:
m_fp.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Defina a estrutura do seu modelo
# Exemplo:
# model = create_model()

# Carregar os arquivos de checkpoint usando tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'))

# Obtendo o modelo do checkpoint
model = checkpoint.model

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

INFO:tensorflow:Assets written to: /home/rodrigo/Documents/neural-audio-fp/guardarCheks/assets


INFO:tensorflow:Assets written to: /home/rodrigo/Documents/neural-audio-fp/guardarCheks/assets


In [ ]:
# Carregar o último ponto de verificação (checkpoint)
latest_checkpoint = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')
model = load_model(latest_checkpoint)

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

In [17]:
checkpoint_dir = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
#os.listdir(checkpoint_dir)

In [7]:
latest = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [8]:
model = get_fingerprinter(cfg, trainable=True)

In [47]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

In [51]:
model.compile(optimizer='ADAM', loss=loss_obj_train, metrics=['accuracy', 'loss_weights='])

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model.load_weights(latest)